In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Function to read stock data from an Excel file
def get_stock_data_from_excel(file_path):
    all_stock_data = pd.read_excel(file_path, sheet_name=None)
    stock_data = {}

    for sheet_name, data in all_stock_data.items():
        # Ensure 'Date' is treated as a datetime column and set as index
        data['Date'] = pd.to_datetime(data['Date'])
        data.set_index('Date', inplace=True)
        stock_data[sheet_name] = data

    return stock_data

In [ ]:
def create_scrip_data(df, N, P):
    N_half = N // 2

    # Initialize the Scrip_Data DataFrame with required columns
    scrip_data = pd.DataFrame(index=df.index)
    scrip_data['Close'] = df['Close']
    scrip_data['Open'] = df['Open']
    scrip_data['High'] = df['High']
    scrip_data['Low'] = df['Low']
    scrip_data['Volume'] = df['Volume']

    # Calculate Change
    scrip_data['Change'] = scrip_data['Close'].diff()

    # Calculate Gain and Loss
    scrip_data['Gain'] = np.where(scrip_data['Change'] > 0, scrip_data['Change'], 0)
    scrip_data['Loss'] = np.where(scrip_data['Gain'] == 0, -1 * scrip_data['Change'], 0)

    # Calculate Average Gain and Average Loss
    scrip_data['Average Gain'] = pd.Series(np.nan, index=scrip_data.index)
    scrip_data['Average Loss'] = pd.Series(np.nan, index=scrip_data.index)

    # Initial Average Gain and Loss for the first P days
    scrip_data.loc[scrip_data.index[P], 'Average Gain'] = scrip_data.loc[scrip_data.index[1:P+1], 'Gain'].mean()
    scrip_data.loc[scrip_data.index[P], 'Average Loss'] = scrip_data.loc[scrip_data.index[1:P+1], 'Loss'].mean()

    # Copy columns to avoid SettingWithCopyWarning
    scrip_data['Average Gain'] = scrip_data['Average Gain'].copy()
    scrip_data['Average Loss'] = scrip_data['Average Loss'].copy()

    # Calculate Average Gain and Average Loss using exponential smoothing formula
    for i in range(P + 1, len(scrip_data)):
        scrip_data.loc[scrip_data.index[i], 'Average Gain'] = ((scrip_data.loc[scrip_data.index[i - 1], 'Average Gain'] * (P - 1)) + scrip_data.loc[scrip_data.index[i], 'Gain']) / P
        scrip_data.loc[scrip_data.index[i], 'Average Loss'] = ((scrip_data.loc[scrip_data.index[i - 1], 'Average Loss'] * (P - 1)) + scrip_data.loc[scrip_data.index[i], 'Loss']) / P


    # Calculate Relative Strength (RS)
    scrip_data['Relative Strength'] = scrip_data['Average Gain'] / scrip_data['Average Loss']

    # Calculate Relative Strength Index (RSI)
    scrip_data['RSI'] = 100 - (100 / (1 + scrip_data['Relative Strength']))

    #Initialize columns for Oversold and Overbought Indicators
    scrip_data['Oversold Indicator'] = pd.Series(np.nan, index=scrip_data.index, dtype='object')
    #scrip_data['Overbought Indicator'] = pd.Series(np.nan, index=scrip_data.index, dtype='object')

    # Initialize the Trade-related columns
    scrip_data[f'Trade taken in last {N_half} days'] = pd.NA
    scrip_data['Enter Trade'] = ''
    scrip_data['Trade Number'] = pd.NA
    scrip_data[f'{N_half}th Day'] = pd.NA

    trade_number = 0
    trade_taken = [0] * len(scrip_data)

    # Modify Trade signals and tracking logic
    for i in range(max(N, P + 1), len(scrip_data)):
        # Check the last N_half days for OB or OS signals
        #last_n_half_days_OB = scrip_data['Overbought Indicator'].iloc[i - N_half:i]
        last_n_half_days_OS = scrip_data['Oversold Indicator'].iloc[i - N_half:i]

        last_n_half_days = scrip_data['Enter Trade'].iloc[i-N_half:i]
        trade_taken[i] = last_n_half_days[last_n_half_days == 'ENTER'].count()

        # No OB or OS signal in the last N_half days
        #ob_signal_in_last_n_half_days = (last_n_half_days_OB == 'OB').any()
        os_signal_in_last_n_half_days = (last_n_half_days_OS == 'OS').any()

        # Generate Oversold and Overbought indicators
        if scrip_data['RSI'].iloc[i] < 30:
            scrip_data.loc[scrip_data.index[i], 'Oversold Indicator'] = 'OS'
        else:
            scrip_data.loc[scrip_data.index[i], 'Oversold Indicator'] = 'NA'

        #if scrip_data['RSI'].iloc[i] > 70:
        #    scrip_data.loc[scrip_data.index[i], 'Overbought Indicator'] = 'OB'
        #else:
        #    scrip_data.loc[scrip_data.index[i], 'Overbought Indicator'] = 'NA'

        # Enter trade logic: Enter only if there is no OB or OS signal in the last N_half days
        if scrip_data['Oversold Indicator'].iloc[i] == 'OS' and not os_signal_in_last_n_half_days:
            scrip_data.loc[scrip_data.index[i], 'Enter Trade'] = 'ENTER'
            trade_number += 1
        else:
            scrip_data.loc[scrip_data.index[i], 'Enter Trade'] = 'NA'

        scrip_data.loc[scrip_data.index[i], 'Trade Number'] = trade_number
        scrip_data.loc[scrip_data.index[i], f'Trade taken in last {N_half} days'] = trade_taken[i]

        # Set the N//2th Day column
        if i + N_half - 1 < len(scrip_data):
                scrip_data.loc[scrip_data.index[i], f'{N_half}th Day'] = scrip_data.index[i + N_half - 1]

        # Update the trade_taken list
        if scrip_data.loc[scrip_data.index[i], 'Enter Trade'] == 'ENTER':
            trade_taken[i] = 1
        else:
            trade_taken[i] = 0

    return scrip_data

In [ ]:
def create_trade_data(scrip_data, N):
    trade_data = []
    N_half = N // 2
    trades = scrip_data[scrip_data['Enter Trade'] == 'ENTER']

    for index, trade in trades.iterrows():
        trade_number = trade['Trade Number']
        entry_day = index
        nth_day = trade[f'{N_half}th Day']
        entry_price = trade['Close']

        # Adjust entry to be at the open of the next day
        next_day_index = scrip_data.index.get_loc(entry_day) + 1
        if next_day_index >= len(scrip_data):
            continue  # Skip if the next day is out of bounds

        next_day_data = scrip_data.iloc[next_day_index]
        entry_day = next_day_data.name  # Update entry_day to the next day
        entry_price = next_day_data['Open']

        # Calculate the N/2th day based on the adjusted entry day
        nth_day_index = next_day_index + (N_half - 1)
        if nth_day_index >= len(scrip_data):
            continue  # Skip if N_halfth day is out of bounds

        nth_day = scrip_data.index[nth_day_index]

        # Include the high for both entry and exit days in the calculation
        trade_period = scrip_data.loc[entry_day:nth_day]
        highest_point = max(trade_period['High'].max(), next_day_data['High'])
        lowest_point = min(trade_period['Low'].min(), next_day_data['Low'])

        highest_point_date = trade_period['High'].idxmax()
        lowest_point_date = trade_period['Low'].idxmin()

        long_potential = ((highest_point - entry_price) / entry_price) * 100
        short_potential = ((entry_price - lowest_point) / entry_price) * 100

        whipsaw_long = 'Yes' if long_potential < 0 else 'No'
        whipsaw_short = 'Yes' if short_potential < 0 else 'No'

        reached_first = 'Lowest' if lowest_point_date < highest_point_date else 'Highest'

        trade_data.append({
            'Trade Number': trade_number,
            'Entry Day': entry_day,  # Now reflects the next day
            f'{N_half}th Day': nth_day,  # Adjusted N_halfth day
            'Entry Price': entry_price,
            'Highest Point': highest_point,
            'Lowest Point': lowest_point,
            'Long Potential': long_potential,
            'Short Potential': short_potential,
            'Whipsaw Long': whipsaw_long,
            'Whipsaw Short': whipsaw_short,
            'Date of Highest Point': highest_point_date,
            'Date of Lowest Point': lowest_point_date,
            'Reached First?': reached_first
        })

    trade_df = pd.DataFrame(trade_data)

    return trade_df


In [ ]:
def create_analysis(trade_data, N):
    N_half = N // 2

    # Calculate required metrics for the Analysis dataframe
    total_trades = len(trade_data)

    long_potential_30th_percentile = trade_data['Long Potential'].quantile(0.30)
    short_potential_30th_percentile = trade_data['Short Potential'].quantile(0.30)

    long_whipsaws_count = trade_data['Whipsaw Long'].value_counts().get('Yes', 0)
    short_whipsaws_count = trade_data['Whipsaw Short'].value_counts().get('Yes', 0)

    short_reached_first = trade_data['Reached First?'].value_counts().get('Highest', 0)
    long_reached_first = trade_data['Reached First?'].value_counts().get('Lowest', 0)

    # Create a dictionary to store the analysis data
    analysis_data = {
        'LookBack Period': N,
        'N/2': N_half,
        'Total Trades': total_trades,
        '30th Percentile Long Potential': long_potential_30th_percentile,
        '30th Percentile Short Potential': short_potential_30th_percentile,
        'Long Whipsaws Count': long_whipsaws_count,
        'Short Whipsaws Count': short_whipsaws_count,
        'Short Reached First': short_reached_first,
        'Long Reached First': long_reached_first
    }

    # Convert the dictionary to a DataFrame
    analysis_df = pd.DataFrame([analysis_data])

    return analysis_df

In [ ]:
# Path to the Excel file containing stock data
file_path = '/content/drive/MyDrive/Data/DATA_FOR_TESTING.xlsx'
output_file_path = '/content/drive/MyDrive/Analysis/Relative Strength Index (Entry at the Open)/AllStocks_Analysis_RSI_Oversold.xlsx'

In [ ]:
# Get stock data from Excel file
stock_data = get_stock_data_from_excel(file_path)

In [ ]:
with pd.ExcelWriter(output_file_path) as writer:
    for scrip, df in stock_data.items():
        print(f"Processing {scrip}...")
        # Create an Excel sheet for each scrip
        analysis_results = []

        for N in range(5, 51):  # Looping N from 5 to 50
            scrip_data = create_scrip_data(df, N=N, P=14)
            trade_data = create_trade_data(scrip_data, N=N)
            analysis_data = create_analysis(trade_data, N=N)
            analysis_results.append(analysis_data)

        # Combine all analysis results for current scrip into a single DataFrame
        combined_analysis_df = pd.concat(analysis_results, ignore_index=True)

        # Save the combined analysis results for the current scrip
        combined_analysis_df.to_excel(writer, sheet_name=scrip, index=False)

Processing AARTIIND...
Processing ABB...
Processing ABBOTINDIA...
Processing ABCAPITAL...
Processing ABFRL...
Processing ACC...
Processing ADANIENT...
Processing ADANIPORTS...
Processing ALKEM...
Processing AMBUJACEM...
Processing APOLLOHOSP...
Processing APOLLOTYRE...
Processing ASHOKLEY...
Processing ASIANPAINT...
Processing ASTRAL...
Processing ATUL...
Processing AUBANK...
Processing AUROPHARMA...
Processing AXISBANK...
Processing BAJAJ-AUTO...
Processing BAJAJFINSV...
Processing BAJFINANCE...
Processing BALKRISIND...
Processing BALRAMCHIN...
Processing BANDHANBNK...
Processing BANKBARODA...
Processing BATAINDIA...
Processing BEL...
Processing BERGEPAINT...
Processing BHARATFORG...
Processing BHARTIARTL...
Processing BHEL...
Processing BIOCON...
Processing BOSCHLTD...
Processing BPCL...
Processing BRITANNIA...
Processing BSOFT...
Processing CANBK...
Processing CANFINHOME...
Processing CHAMBLFERT...
Processing CHOLAFIN...
Processing CIPLA...
Processing COALINDIA...
Processing COFORGE

In [ ]:
print(f"Processed data saved to {output_file_path}")

Processed data saved to /content/drive/MyDrive/Analysis/Relative Strength Index (Entry at the Open)/AllStocks_Analysis_RSI_Oversold.xlsx
